# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
J. Liu  ->  J. Liu  |  ['J. Liu']
F. Briegel  ->  F. Briegel  |  ['F. Briegel']
M. Kuhlberg  ->  M. Kuhlberg  |  ['M. Kuhlberg']
S. Savvidou  ->  S. Savvidou  |  ['S. Savvidou']


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Arxiv has 58 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.07940


extracting tarball to tmp_2407.07940...

 done.


Found 92 bibliographic references in tmp_2407.07940/aanda.bbl.
syntax error in line 368: '=' expected
Retrieving document from  https://arxiv.org/e-print/2407.07965
extracting tarball to tmp_2407.07965...

...

/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Intensity_plot
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure Intensity_plot as tmp_2407.08533/./Intensity_plot.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 79 bibliographic references in tmp_2407.08533/SavvidouBitsch24.bbl.
Retrieving document from  https://arxiv.org/e-print/2407.08643


extracting tarball to tmp_2407.08643...

 done.


Found 154 bibliographic references in tmp_2407.08643/mnras_template.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.07940-b31b1b.svg)](https://arxiv.org/abs/2407.07940) | **Euclid preparation. Forecasting the recovery of galaxy physical properties and their relations with template-fitting and machine-learning methods**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-07-12*|
|*Comments*| *26 pages, 13 figures. Submitted to A&A*|
|**Abstract**|            Euclid will collect an enormous amount of data during the mission's lifetime, observing billions of galaxies in the extragalactic sky. Along with traditional template-fitting methods, numerous Machine Learning algorithms have been presented for computing their photometric redshifts and physical parameters (PP), requiring significantly less computing effort while producing equivalent performance measures. However, their performance is limited by the quality and amount of input information, to the point where the recovery of some well-established physical relationships between parameters might not be guaranteed. To forecast the reliability of Euclid photo-$z$s and PPs calculations, we produced two mock catalogs simulating Euclid photometry. We simulated the Euclid Wide Survey (EWS) and Euclid Deep Fields (EDF). We tested the performance of a template-fitting algorithm (Phosphoros) and four ML methods in recovering photo-$z$s, stellar masses, star-formation rates, and the SFMS. To mimic the Euclid processing as closely as possible, the models were trained with Phosphoros-recovered labels. For the EWS, we found that the best results are achieved with a Mixed Labels approach, training the models with Wide survey features and labels from the Phosphoros results on deeper photometry, i.e., with the best possible set of labels for a given photometry. This imposes a prior, helping the models to better discern cases in degenerate regions of feature space, i.e., when galaxies have similar magnitudes and colors but different redshifts and PPs, with performance metrics even better than those found with Phosphoros. We found no more than $3\%$ performance degradation using a COSMOS-like reference sample or removing $u$ band data, which will not be available until after data release DR1. The best results are obtained for the EDF, with appropriate recovery of photo-$z$, PPs, and the SFMS.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.08319-b31b1b.svg)](https://arxiv.org/abs/2407.08319) | **Telescope control software and proto-model siderostat for the SDSS-V Local Volume Mapper**  |
|| H. Ahn, et al. -- incl., <mark>F. Briegel</mark>, <mark>M. Kuhlberg</mark> |
|*Appeared on*| *2024-07-12*|
|*Comments*| *29 pages, 15 figures, in press at Journal of Astronomical Telescopes, Instruments, and Systems*|
|**Abstract**|            The fifth Sloan Digital Sky Survey (SDSS-V) Local Volume Mapper (LVM) is a wide-field integral field unit (IFU) survey that uses an array of four 160 mm fixed telescopes with siderostats to minimize the number of moving parts. Individual telescope observes the science field or calibration field independently and is synchronized with the science exposure. We developed the LVM Acquisition and Guiding Package (LVMAGP) optimized telescope control software program for LVM observations, which can simultaneously control four focusers, three K-mirrors, one fiber selector, four mounts (siderostats), and seven guide cameras. This software is built on a hierarchical architecture and the SDSS framework and provides three key sequences: autofocus, field acquisition, and autoguide. We designed and fabricated a proto-model siderostat to test the telescope pointing model and LVMAGP software. The mirrors of the proto-model were designed as an isogrid open-back type, which reduced the weight by 46% and enabled reaching thermal equilibrium quickly. Additionally, deflection due to bolting torque, self-gravity, and thermal deformation was simulated, and the maximum scatter of the pointing model induced by the tilt of optomechanics was predicted to be $4'.4$, which can be compensated for by the field acquisition sequence. We performed a real sky test of LVMAGP with the proto-model siderostat and obtained field acquisition and autoguide accuracies of $0''.38$ and $1''.5$, respectively. It met all requirements except for the autoguide specification, which will be resolved by more precise alignment among the hardware components at Las Campanas Observatory.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.08533-b31b1b.svg)](https://arxiv.org/abs/2407.08533) | **There is no disk mass budget problem of planet formation**  |
|| <mark>S. Savvidou</mark>, B. Bitsch |
|*Appeared on*| *2024-07-12*|
|*Comments*| *8 pages, 4 figures, submitted to A&A*|
|**Abstract**|            The inferred dust masses from Class II protoplanetary disk observations are lower than or equal to the masses of the observed exoplanet systems. This poses the question of how planets form if their natal environments do not contain enough mass. This hypothesis has entered the literature as the "mass budget problem" of planet formation. We utilize numerical simulations of planet formation via pebble and gas accretion, including migration, in a viscously evolving protoplanetary disk, while tracing the time evolution of the dust mass. As expected, we find that the presence of a giant planet in the disk can influence the evolution of the disk itself and prevent rapid dust mass loss by trapping the dust outside its orbit. Early formation is crucial for giant planet formation, as we found in our previous work; therefore, our findings strengthen the hypothesis that planet formation has already occurred or is ongoing in Class II disks. Most importantly, we find that the optically thin dust mass significantly underestimates the total dust mass in the presence of a dust-trapping deep gap. We also show that the beam convolution would smear out the feature from a deep gap, especially if the planet forms in the inner disk. Such hidden dust mass, along with early planet formation, could be the answer to the hypothetical mass budget problem.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.08643-b31b1b.svg)](https://arxiv.org/abs/2407.08643) | **JADES -- The Rosetta Stone of JWST-discovered AGN: deciphering the intriguing nature of early AGN**  |
|| I. Juodžbalis, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-07-12*|
|*Comments*| *21 pages 8 figures in main text. Submitted to MNRAS*|
|**Abstract**|            JWST has discovered a large population of Active Galactic Nuclei (AGN) at high redshift. Many of these newly discovered AGN have broad permitted lines (typically H$\alpha$), but are extremely weak in the X-rays. Here we present the NIRSpec spectrum of the most extreme of these objects, GN-28074, an AGN at $z=2.26$ with prominent Balmer, Paschen and \HeI broad lines, and with the highest limit on the bolometric to X-ray luminosity ratio among all spectroscopically confirmed AGN in GOODS. This source is also characterized by a mid-IR excess, most likely associated with the AGN torus' hot dust. The high bolometric luminosity and moderate redshift of this AGN allow us to explore its properties more in depth relative to other JWST-discovered AGN. The NIRSpec spectrum reveals prominent, slightly blueshifted absorption of H$\alpha$, H$\beta$ and \HeI$\lambda$10830. The Balmer absorption lines require gas with densities of $n_{\rm H}> 10^8~{\rm cm}^{-3}$, inconsistent with an ISM origin, but fully consistent with clouds in the Broad Line Region (BLR). This finding suggests that at least part of the X-ray weakness is due to high (Compton thick) X-ray absorption by (dust-free) clouds in the BLR, or in its outer, slowly outflowing regions. GN-28074 is also extremely radio-weak. The radio weakness can also be explained in terms of absorption, as the inferred density of the clouds responsible for H$\alpha$ absorption makes them optically thick to radio emission through free-free absorption. Alternatively, in this and other JWST-discovered AGN, the nuclear magnetic field may have not developed properly yet, resulting both in intrinsically weak radio emission and also lack of hot corona, hence intrinsic X-ray weakness. Finally, we show that recently proposed scenarios, invoking hyper-dense and ultra-metal-poor outflows or Raman scattering to explain the broad H$\alpha$, are completely ruled out.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.07965-b31b1b.svg)](https://arxiv.org/abs/2407.07965) | **Sulphur dioxide in the mid-infrared transmission spectrum of WASP-39b**  |
|| D. Powell, et al. -- incl., <mark>X. Zhang</mark>, <mark>E.-M. Ahrer</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-07-12*|
|*Comments*| *Published in Nature*|
|**Abstract**|            The recent inference of sulphur dioxide (SO$_2$) in the atmosphere of the hot ($\sim$1100 K), Saturn-mass exoplanet WASP-39b from near-infrared JWST observations suggests that photochemistry is a key process in high temperature exoplanet atmospheres. This is due to the low ($<$1 ppb) abundance of SO$_2$ under thermochemical equilibrium, compared to that produced from the photochemistry of H$_2$O and H$_2$S (1-10 ppm). However, the SO$_2$ inference was made from a single, small molecular feature in the transmission spectrum of WASP-39b at 4.05 $\mu$m, and therefore the detection of other SO$_2$ absorption bands at different wavelengths is needed to better constrain the SO$_2$ abundance. Here we report the detection of SO$_2$ spectral features at 7.7 and 8.5 $\mu$m in the 5-12 $\mu$m transmission spectrum of WASP-39b measured by the JWST Mid-Infrared Instrument (MIRI) Low Resolution Spectrometer (LRS). Our observations suggest an abundance of SO$_2$ of 0.5-25 ppm (1$\sigma$ range), consistent with previous findings. In addition to SO$_2$, we find broad water vapour absorption features, as well as an unexplained decrease in the transit depth at wavelengths longer than 10 $\mu$m. Fitting the spectrum with a grid of atmospheric forward models, we derive an atmospheric heavy element content (metallicity) for WASP-39b of $\sim$7.1-8.0 $\times$ solar and demonstrate that photochemistry shapes the spectra of WASP-39b across a broad wavelength range.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.08119-b31b1b.svg)](https://arxiv.org/abs/2407.08119) | **Improving the Automated Coronal Jet Identification with U-NET**  |
|| <mark>J. Liu</mark>, et al. |
|*Appeared on*| *2024-07-12*|
|*Comments*| *13 pages, 7 figures*|
|**Abstract**|            Coronal jets are one of the most common eruptive activities in the solar atmosphere. They are related to rich physics processes, including but not limited to magnetic reconnection, flaring, instabilities, and plasma heating. Automated identification of off-limb coronal jets has been difficult due to their abundant nature, complex appearance, and relatively small size compared to other features in the corona. In this paper, we present an automated coronal jet identification algorithm (AJIA) that utilizes true and fake jets previously detected by a laborious semi-automated jet detection algorithm (SAJIA, Liu et al. 2023) as the input of an image segmentation neural network U-NET. It is found that AJIA could achieve a much higher (0.81) detecting precision than SAJIA (0.34), meanwhile giving the possibility of whether each pixel in an input image belongs to a jet. We demonstrate that with the aid of artificial neural networks, AJIA could enable fast, accurate, and real-time coronal jet identification from SDO/AIA 304 Åobservations, which are essential in studying the collective and long-term behavior of coronal jets and their relation with the solar activity cycles.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.07940/./images/WIDE/Photoz.png', 'tmp_2407.07940/./images/WIDE/SFR.png', 'tmp_2407.07940/./images/WIDE/StellarMass.png']
copying  tmp_2407.07940/./images/WIDE/Photoz.png to _build/html/
copying  tmp_2407.07940/./images/WIDE/SFR.png to _build/html/
copying  tmp_2407.07940/./images/WIDE/StellarMass.png to _build/html/
exported in  _build/html/2407.07940.md
    + _build/html/tmp_2407.07940/./images/WIDE/Photoz.png
    + _build/html/tmp_2407.07940/./images/WIDE/SFR.png
    + _build/html/tmp_2407.07940/./images/WIDE/StellarMass.png
found figures ['tmp_2407.08319/./architecture_allinst.png', 'tmp_2407.08319/./siderostat_design.png', 'tmp_2407.08319/./FEA_optomech.png']
copying  tmp_2407.08319/./architecture_allinst.png to _build/html/
copying  tmp_2407.08319/./siderostat_design.png to _build/html/
copying  tmp_2407.08319/./FEA_optomech.png to _build/html/
exported in  _build/html/2407.08319.md
    + _build/html/tmp_2407.08319/./architecture_allinst.png
    + _build/h

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mambo}{\texttt{Mambo}}$
$\newcommand{\nnpz}{\texttt{nnpz}}$
$\newcommand{\phosphoros}{\texttt{Phosphoros}}$
$\newcommand{\catboost}{\texttt{CatBoost}}$
$\newcommand{\sfr}{\logten \mathrm{SFR}}$
$\newcommand{\Mstar}{\logten M_{\star}}$
$\newcommand{\logMsun}{\logten M_\odot}$
$\newcommand{\logMsunyr}{\logten(M_\odot \mathrm{yr}^{-1})}$
$\newcommand{\Mstarwun}{\logten(M_{\star}/M_\odot)}$
$\newcommand{\sfrwun}{\logten (\mathrm{SFR}/M_\odot \mathrm{yr}^{-1})}$
$\newcommand{\afme}[1]{ {\color{teal}(\it AE: #1)}}$
$\newcommand{\MB}[1]{ {\color{orange}(MB: #1)}}$
$\newcommand{\LP}[1]{ {\color{magenta}(LP: #1)}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$\/ preparation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.07940-b31b1b.svg)](https://arxiv.org/abs/2407.07940)<mark>Appeared on: 2024-07-12</mark> -  _26 pages, 13 figures. Submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** $\Euclid$ will collect an enormous amount of data during the mission's lifetime, observing billions of galaxies in the extragalactic sky. Along with traditional template-fitting methods, numerous Machine Learning (ML) algorithms have been presented for computing their photometric redshifts and physical parameters (PP), requiring significantly less computing effort while producing equivalent performance measures. However, their performance is limited by the quality and amount of input information entering the model (the $_ features_$ ), to a level where the recovery of some well-established physical relationships between parameters might not be guaranteed (e.g., the star-forming main sequence, SFMS).   To forecast the reliability of $\Euclid$ photo- $z$ s and PPs calculations, we produced two mock catalogs simulating the photometry with the UNIONS $ugriz$ and $\Euclid$ filters. We simulated the Euclid Wide Survey (EWS) and Euclid Deep Fields (EDF), alongside two auxiliary fields. We tested the performance of a template-fitting algorithm ( $\phosphoros$ ) and four ML methods in recovering photo- $z$ s, PPs (stellar masses and star-formation rates), and the SFMS on the simulated $\Euclid$ fields. To mimic the $\Euclid$ processing as closely as possible, the models were trained with $\phosphoros$ -recovered labels and tested on the simulated ground truth. For the EWS, we found that the best results are achieved with a Mixed Labels approach, training the models with Wide survey features and labels from the $\phosphoros$ results on deeper photometry, i.e., with the best possible set of labels for a given photometry. This imposes a prior to the input features, helping the models to better discern cases in degenerate regions of feature space, i.e., when galaxies have similar magnitudes and colors but different redshifts and PPs, with performance metrics even better than those found with $\phosphoros$ . We found no more than 3 \% performance degradation using a COSMOS-like reference sample or removing $u$ band data, which will not be available until after data release DR1. The best results are obtained for the EDF, with appropriate recovery of photo- $z$ , PPs, and the SFMS.

</div>

<div id="div_fig1">

<img src="tmp_2407.07940/./images/WIDE/Photoz.png" alt="Fig7" width="100%"/>

**Figure 7. -** Results for the \gls{EWS} with the _ Mixed Labels_ approach. The true values on the $x$-axis are plotted against the predicted values on $y$. The black line is the 1:1 relation; the shaded area is the region beyond which a prediction is an outlier. Contours are the area containing $98\%$, $86\%$, $39\%$(corresponding to the $3\sigma$, $2\sigma$ and $1\sigma$ levels for a 2D histogram) and $20\%$ of the sample. Each column represents the results for the methods described in Sect. \ref{sec:Methodology}. In the first four rows, the training labels are the recovered ones, coming from $\phosphoros$ results to the mock photometry at the same depth of the field reported in the leftmost plot legend and tested on the \gls{EWS}(see Sect. \ref{sec:Mocks} for further details). The T.lab Wide-Wide case is exactly the same as the Wide-Wide case in Table \ref{tab:WideRec}. In the fifth row, we show the results of the \gls{EWS} training the models with their true labels as the best-case scenario for that particular field. The reported metrics are NMAD (purple), the outlier fraction $f_{\rm out}$(blue) and the bias (green) for the photometric redshifts and physical parameters, as well as the slope $m$, scatter $\sigma$ and fraction of passive galaxies $f_{\rm p}$ for the \gls{ms}, all defined in Sect. \ref{sec:metrics}). (*fig:WIDE_zphot*)

</div>
<div id="div_fig2">

<img src="tmp_2407.07940/./images/WIDE/SFR.png" alt="Fig9" width="100%"/>

**Figure 9. -** Same as in Fig. \ref{fig:WIDE_zphot}, for star-formation rates. (*fig:WIDE_SFR*)

</div>
<div id="div_fig3">

<img src="tmp_2407.07940/./images/WIDE/StellarMass.png" alt="Fig8" width="100%"/>

**Figure 8. -** Same as in Fig. \ref{fig:WIDE_zphot}, for stellar masses. (*fig:WIDE_M*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.07940"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cftdotsep}{\cftnodots}$</div>



<div id="title">

# Telescope control software and proto-model siderostat for the SDSS-V Local Volume Mapper

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.08319-b31b1b.svg)](https://arxiv.org/abs/2407.08319)<mark>Appeared on: 2024-07-12</mark> -  _29 pages, 15 figures, in press at Journal of Astronomical Telescopes, Instruments, and Systems_

</div>
<div id="authors">

H. Ahn, et al. -- incl., <mark>F. Briegel</mark>, <mark>M. Kuhlberg</mark>

</div>
<div id="abstract">

**Abstract:** The fifth Sloan Digital Sky Survey (SDSS-V) Local Volume Mapper (LVM) is a wide-field integral field unit (IFU) survey that uses an array of four 160 mm fixed telescopes with siderostats to minimize the number of moving parts. Individual telescope observes the science field or calibration field independently and is synchronized with the science exposure. We developed the LVM Acquisition and Guiding Package (LVMAGP) optimized telescope control software program for LVM observations, which can simultaneously control four focusers, three K-mirrors, one fiber selector, four mounts (siderostats), and seven guide cameras. This software is built on a hierarchical architecture and the SDSS framework and provides three key sequences: autofocus, field acquisition, and autoguide. We designed and fabricated a proto-model siderostat to test the telescope pointing model and LVMAGP software. The mirrors of the proto-model were designed as an isogrid open-back type, which reduced the weight by 46 \% and enabled reaching thermal equilibrium quickly. Additionally, deflection due to bolting torque, self-gravity, and thermal deformation was simulated, and the maximum scatter of the pointing model induced by the tilt of optomechanics was predicted to be $4'.4$ , which can be compensated for by the field acquisition sequence. We performed a real sky test of LVMAGP with the proto-model siderostat and obtained field acquisition and autoguide accuracies of $0".38$ and $1".5$ , respectively. It met all requirements except for the autoguide specification, which will be resolved by more precise alignment among the hardware components at Las Campanas Observatory.

</div>

<div id="div_fig1">

<img src="tmp_2407.08319/./architecture_allinst.png" alt="Fig2" width="100%"/>

**Figure 2. -** System architecture, including all individual instruments and ICS instances under the telescope subsystem
    System architecture, including all individual instruments and ICS instances under the telescope subsystem. Each system element is denoted in the format "telescope unit.element." Connections between components are illustrated by lines, with the type of connection indicated by the color of line. The chrome yellow oval indicates the container where the software runs. LVMAGP communicates with controllers in containers C4 to C11, although the arrow only points to the spec.LVMTAN actor to simplify the figure.
     (*fig:architecture_allinst*)

</div>
<div id="div_fig2">

<img src="tmp_2407.08319/./siderostat_design.png" alt="Fig7" width="100%"/>

**Figure 7. -** 3D mechanical design of the proto-model siderostat in 3D view
    3D mechanical design of the proto-model siderostat in 3D view (left) with the mount and (right) without the mount. The SAz and SEl mirrors are colored green and indicated by black arrows. The mirror holders (blue), adapters (orange), and base module (gray) are all assembled. The proto-model siderostat is in the home position, where it points to the zenith. In the left panel, the light path is expressed with light entering direction and indicated with a red arrow. In the right panel, the aluminum profile part is indicated with a black arrow. (*fig:siderostat_design*)

</div>
<div id="div_fig3">

<img src="tmp_2407.08319/./FEA_optomech.png" alt="Fig9" width="100%"/>

**Figure 9. -** Stress and displacement of the proto-model siderostat at the zenith position at the temperature of $-5^\circ$C(left) Loaded von Mises stress and (right) displacement of the proto-model siderostat at the zenith position at the temperature of $-5^\circ$C. The figures are 300 times more exaggerated than the actual variations. The maximum stress point is indicated in the left panel; however, this point is a stress singularity and should be excluded from the evaluation of FoS. Actual maximum stress is applied by bolting torque on section A-A$'$, where the wedge and mount are connected by bolting. (*fig:FEA_optomech*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.08319"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\be}{\begin{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\notes}[1]{\textcolor{violet}{\textit{#1}}}$
$\title{There is no disk mass budget problem of planet formation}$
$\author{Sofia Savvidou \inst{1}$
$\and Bertram Bitsch \inst{2,1}}$
$\offprints{S. Savvidou,\ \email{savvidou@mpia.de}}$
$\institute{$
$Max-Planck-Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany$
$\and$
$Department of Physics, University College Cork, Cork, Ireland}$
$\date{Received date / Accepted date }$
$\begin{document}$
$\abstract{The inferred dust masses from Class II protoplanetary disk observations are lower than or equal to the masses of the observed exoplanet systems. This poses the question of how planets form if their natal environments do not contain enough mass. This hypothesis has entered the literature as the "mass budget problem" of planet formation. We utilize numerical simulations of planet formation via pebble and gas accretion, including migration, in a viscously evolving protoplanetary disk, while tracing the time evolution of the dust mass. As expected, we find that the presence of a giant planet in the disk can influence the evolution of the disk itself and prevent rapid dust mass loss by trapping the dust outside its orbit. Early formation is crucial for giant planet formation, as we found in our previous work; therefore, our findings strengthen the hypothesis that planet formation has already occurred or is ongoing in Class II disks. Most importantly, we find that the optically thin dust mass significantly underestimates the total dust mass in the presence of a dust-trapping deep gap. We also show that the beam convolution would smear out the feature from a deep gap, especially if the planet forms in the inner disk. Such hidden dust mass, along with early planet formation, could be the answer to the hypothetical mass budget problem.}$
$\keywords{protoplanetary disks -- planets and satellites: formation, gaseous planets -- circumstellar matter -- methods: numerical}$
$\maketitle$
$\n\end{document}\end{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\notes}[1]{\textcolor{violet}{\textit{#1}}}$</div>



<div id="title">

# There is no disk mass budget problem of planet formation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.08533-b31b1b.svg)](https://arxiv.org/abs/2407.08533)<mark>Appeared on: 2024-07-12</mark> -  _8 pages, 4 figures, submitted to A&A_

</div>
<div id="authors">

<mark>S. Savvidou</mark>, B. Bitsch

</div>
<div id="abstract">

**Abstract:** The inferred dust masses from Class II protoplanetary disk observations are lower than or equal to the masses of the observed exoplanet systems. This poses the question of how planets form if their natal environments do not contain enough mass. This hypothesis has entered the literature as the "mass budget problem" of planet formation. We utilize numerical simulations of planet formation via pebble and gas accretion, including migration, in a viscously evolving protoplanetary disk, while tracing the time evolution of the dust mass. As expected, we find that the presence of a giant planet in the disk can influence the evolution of the disk itself and prevent rapid dust mass loss by trapping the dust outside its orbit. Early formation is crucial for giant planet formation, as we found in our previous work; therefore, our findings strengthen the hypothesis that planet formation has already occurred or is ongoing in Class II disks. Most importantly, we find that the optically thin dust mass significantly underestimates the total dust mass in the presence of a dust-trapping deep gap. We also show that the beam convolution would smear out the feature from a deep gap, especially if the planet forms in the inner disk. Such hidden dust mass, along with early planet formation, could be the answer to the hypothetical mass budget problem.

</div>

<div id="div_fig1">

<img src="tmp_2407.08533/./Mdust_opticallythin_evolution_main.png" alt="Fig1" width="100%"/>

**Figure 1. -** Dust mass as a function of time for the same planets as in Fig. 1 in [ and Savvidou (2023)](), using the standard set of parameters (Table \ref{Tab:parameters}). The dashed lines show the time evolution of the optically thin dust mass and the red line shows the time evolution of the dust mass for a simulation without a planet. The dot marks the last dust mass if a planet reaches the inner edge of the disk. (*Fig:t-Mdust_main*)

</div>
<div id="div_fig2">

<img src="tmp_2407.08533/./Mdust_opticallythin_evolution.png" alt="Fig3" width="100%"/>

**Figure 3. -** Same as Fig. \ref{Fig:t-Mdust_main}, with one parameter changed (marked in each panel) compared to the standard model (bold in Table \ref{Tab:parameters}). These correspond to the same planets as in Fig. 2 in [ and Savvidou (2023)](). (*Fig:t-Mdust*)

</div>
<div id="div_fig3">

<img src="tmp_2407.08533/./Intensity_plot.png" alt="Fig4" width="100%"/>

**Figure 4. -** Normalized intensity as a function of orbital distance at 1.5 Myr, comparing the uncolvolved intensity with the ones convolved with three different beams of 0.02" and 0.04" at 140 pc (2.8 and 5.6 AU), and 0.34" at 232 pc (100 AU). The gray dashed lines show the evaporation fronts that mainly cause the spikes in the intensity. The examples with a giant correspond to the standard set of parameters (Fig. \ref{Fig:t-Mdust_main}), while the example without a giant corresponds to a low-mass disk (top, left plot in Fig. \ref{Fig:t-Mdust}). (*Fig:r-I*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.08533"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\HII}{H {\sc ii}\rm }$
$\newcommand{\NII}{{[N {\sc ii}]}}$
$\newcommand{\NIIs}{{[N {\sc ii}] }}$
$\newcommand{\NVs}{{[N {\sc V}] }}$
$\newcommand{\NVl}{{N {\sc V} \lambda}}$
$\newcommand{\NIIl}{{[N {\sc ii}] \lambda}}$
$\newcommand{\NeIII}{{[Ne {\sc iii}]}}$
$\newcommand{\NeIIIs}{{[Ne {\sc iii}] }}$
$\newcommand{\NeIIIl}{{[Ne {\sc iii}] \lambda}}$
$\newcommand{\NeIV}{{[Ne {\sc iv}]}}$
$\newcommand{\NeIVs}{{[Ne {\sc iv}] }}$
$\newcommand{\NeIVl}{{[Ne {\sc iv}] \lambda}}$
$\newcommand{\NeV}{{[Ne {\sc v}]}}$
$\newcommand{\NeVs}{{[Ne {\sc v}] }}$
$\newcommand{\NeVl}{{[Ne {\sc v}] \lambda}}$
$\newcommand{\SII}{{[S {\sc ii}]}}$
$\newcommand{\SIIs}{{[S {\sc ii}] }}$
$\newcommand{\SIIl}{{[S {\sc ii}] \lambda}}$
$\newcommand{\SIIll}{{[S {\sc ii}] \lambda\lambda}}$
$\newcommand{\SIII}{{[S {\sc iii}]}}$
$\newcommand{\SIIIs}{{[S {\sc iii}] }}$
$\newcommand{\SIIIl}{{[S {\sc iii}] \lambda}}$
$\newcommand{\OIII}{{[O {\sc iii}]}}$
$\newcommand{\OIV}{{[O {\sc iv}]}}$
$\newcommand{\OIIIs}{{[O {\sc iii}] }}$
$\newcommand{\OIIIl}{{[O {\sc iii}] \lambda}}$
$\newcommand{\OII}{{[O {\sc ii}]}}$
$\newcommand{\OIIs}{{[O {\sc ii}] }}$
$\newcommand{\OIIl}{{[O {\sc ii}] \lambda}}$
$\newcommand{\OIIll}{{[O {\sc ii}] \lambda\lambda}}$
$\newcommand{\OI}{{[O {\sc i}]}}$
$\newcommand{\OIs}{{[O {\sc i}] }}$
$\newcommand{\OIl}{{[O {\sc i}] \lambda}}$
$\newcommand{\CII}{{[C {\sc ii}]}}$
$\newcommand{\CIIs}{{C {\sc ii}] }}$
$\newcommand{\CIII}{{C {\sc iii}]}}$
$\newcommand{\CIIIs}{{C {\sc iii}] }}$
$\newcommand{\CIIIll}{\CIIIs\lambda\lambda}$
$\newcommand{\CIV}{{C {\sc iv}}}$
$\newcommand{\CIVs}{{C {\sc iv} }}$
$\newcommand{\CIVll}{{C {\sc iv} \lambda\lambda}\xspace}$
$\newcommand{\CIVl}{{[C {\sc iv} \lambda}\xspace}$
$\newcommand{\HeII}{{He {\sc ii} }}$
$\newcommand{\HeI}{{He {\sc i} }}$
$\newcommand{\HeIIl}{{He {\sc ii} \lambda}}$
$\newcommand{\FeII}{{[Fe {\sc ii}] }}$
$\newcommand{\Ha}{H\alpha}$
$\newcommand{\Has}{H\alpha }$
$\newcommand{\Hb}{H\beta}$
$\newcommand{\Hg}{H\gamma}$
$\newcommand{\Hbs}{H\beta }$
$\newcommand{\purpletxt}[1]{\textcolor{purple}{#1}}$
$\newcommand{\RMcomm}[1]{{\color{teal}[RM: #1]}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# JADES - The Rosetta Stone of JWST-discovered AGN: deciphering the intriguing nature of early AGN

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.08643-b31b1b.svg)](https://arxiv.org/abs/2407.08643)<mark>Appeared on: 2024-07-12</mark> -  _21 pages 8 figures in main text. Submitted to MNRAS_

</div>
<div id="authors">

I. Juodžbalis, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** JWST has discovered a large population of Active Galactic Nuclei (AGN) at high redshift. Many of these newly discovered AGN have broad permitted lines (typically H $\alpha$ ), but are extremely weak in the X-rays.Here we present the NIRSpec spectrum of the most extreme of these objects, GN-28074, an AGN at $z=2.26$ with prominent Balmer, Paschen and $\HeI$ broad lines, and with the highest limit on the bolometric to X-ray luminosity ratio among all spectroscopically confirmed AGN in GOODS. This source is also characterized by a mid-IR excess, most likely associated with the AGN torus' hot dust. The high bolometric luminosity and moderate redshift of this AGN allow us to explore its properties more in depth relative to other JWST-discovered AGN. The NIRSpec spectrum reveals prominent, slightly blueshifted absorption of H $\alpha$ , H $\beta$ and $\HeI$ $\lambda$ 10830. The Balmer absorption lines require gas with densities of $n_{\rm H}> 10^8 {\rm cm}^{-3}$ , inconsistent with an ISM origin, but fully consistent with clouds in the Broad Line Region (BLR). This finding suggests that at least part of the X-ray weakness is due to high (Compton thick) X-ray absorption by (dust-free) clouds in the BLR, or in its outer, slowly outflowing regions.GN-28074 is also extremely radio-weak. The radio weakness can also be explained in terms of absorption, as the inferred density of the clouds responsible for H $\alpha$ absorption makes them optically thick to radio emission through free-free absorption. Alternatively, in this and other JWST-discovered AGN, the nuclear magnetic field may have not developed properly yet, resulting both in intrinsically weak radio emission and also lack of hot corona, hence intrinsic X-ray weakness. Finally, we show that recently proposed scenarios, invoking hyper-dense and ultra-metal-poor outflows or Raman scattering to explain the broad H $\alpha$ , are completely ruled out.

</div>

<div id="div_fig1">

<img src="tmp_2407.08643/./28074_spec.png" alt="Fig7" width="100%"/>

**Figure 7. -** R1000 spectrum of GN-28074 with emission lines marked. The fluxes on the y axis are presented in _ logarithmic_ scale to better showcase weaker lines and the shape of the continuum. All bright permitted lines appear to have broad components while the continuum showcases the characteristic `v' shape found in some objects of the LRD population. The false-colour NIRCam image (inset) highlights the point-source nature of the object, and the position of the MSA slitlets (grey rectangles). We note a pink region to the south-east, denoting a deficiency in F182M flux (see text for a discussion). The rest of the NIRCam single-band cutouts highlight the extended emission (only discernible at SW wavelengths); the angular size of these cutouts is the same as the RGB image. (*fig:full_spec*)

</div>
<div id="div_fig2">

<img src="tmp_2407.08643/./Absorption_fig/Hbeta_OIII.png" alt="Fig8.1" width="25%"/><img src="tmp_2407.08643/./Absorption_fig/Halpha_SII.png" alt="Fig8.2" width="25%"/><img src="tmp_2407.08643/./Absorption_fig/HeI_PaG.png" alt="Fig8.3" width="25%"/><img src="tmp_2407.08643/./Absorption_fig/PaB_BLR.png" alt="Fig8.4" width="25%"/>

**Figure 8. -** R1000 spectrum expanded around some of the permitted lines of GN-28074 and showing the associated spectral fits. The broad line region (BLR) and the (extended) outflow components are plotted as solid blue and red lines respectively. The narrow components are shown with dashed green lines. The combined fit, including the absorption absorption components, is shown in magenta. **Top left: **$\Hb$s and $\OIII$$\lambda\lambda$5007,4959 complex, showcasing the deep absorption in $\Hb$s and broadening of $\OIII$$\lambda\lambda$5007,4959 lines due to the presence an outflow. **Top right: **$\Ha$s, $\NII$$\lambda\lambda$6548,6583 and $\SII$$\lambda\lambda$6716,6731 . Notable is the relative weakness of the $\NII$$\lambda\lambda$6549,6585 and $\SII$$\lambda\lambda$6716,6731 doublets. **Bottom left: ** Combined $\HeI$$\lambda$10830 and Pa$\gamma$ line fit. The ionized outflow component is not required here by the data. **Bottom right: ** Fit to the Pa$\beta$ line. Due to wavelength calibration issues, this line is slightly offset to the rest and thus requires separate kinematics to properly fit. The ionized outflow is absent just as in the Pa$\gamma$ + $\HeI$$\lambda$10830 fit. (*fig:spectrum_abs*)

</div>
<div id="div_fig3">

<img src="tmp_2407.08643/./hist_Xrays.png" alt="Fig2" width="100%"/>

**Figure 2. -** Distribution of the bolometric to X-ray luminosity ratio, relative to the relation inferred for low-redshift optically-selected AGN (whose distribution is shown with a black, hollow histogram). The distribution of JWST-discovered AGN in GOODS (the vast majority of which are actually lower limits on $L_{Bol}/L_X$) is shown with the solid, amber histogram. The lower limit for GN-280974 is indicated with the blue bar. We also show the lower limit observed in SBS 0335-052E, which is a nearby AGN with similar properties as the new population of the JWST-discovered AGN. We also show the value for the `prototypical' Seyfert 1 NGC-4151 which shows H$\alpha$, H$\beta$ and HeI absorption, just like GN-28074, and strong (and variable) X-ray absorption. (*fig:hist_Xrays*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.08643"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

132  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

22  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

10  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
